In [ ]:
from langchain_groq import ChatGroq
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.callbacks.base import BaseCallbackHandler

class CustomStreamingCallbackHandler(BaseCallbackHandler):
    def __init__(self):
        self.final_output = ""

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        # Stream tokens as they come in
        print(token, end="", flush=True)  # Output directly to the console for real-time streaming
        self.final_output += token

    def get_final_output(self):
        # Return the full response once streaming is complete
        return self.final_output.strip()

class CSVPipeline:
    def __init__(self, llm=None, retriever=None, prompt=None):
        self.llm = llm
        self.retriever = retriever
        self.prompt = prompt
        self.callback_handler = CustomStreamingCallbackHandler()

    def defineLLM(self, key, model_name="Gemma2-9b-It"):
        self.llm = ChatGroq(
            groq_api_key=key,
            model_name=model_name,
            streaming=True,
            callbacks=[self.callback_handler]
        )

    def retrieval(self):
        NEO4J_URI = "neo4j+s://7a76cce7.databases.neo4j.io"
        NEO4J_USERNAME = "neo4j"
        NEO4J_PASSWORD = "iLflxvMlzC8QtxKRYA3cIIEDiR_PzH4vfsLQS08RJXY"
        graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
        self.retriever = GraphCypherQAChain.from_llm(llm=self.llm, graph=graph, verbose=True, allow_dangerous_requests=True)

    def answer(self, prompt):
        system_message = SystemMessagePromptTemplate.from_template("You are a helpful assistant. Please give the exact answers.")
        human_message = HumanMessagePromptTemplate.from_template("{input}")
        chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])
        formatted_prompt = chat_prompt.format_messages(input=prompt)

        if self.retriever is not None:
            self.retriever.run(formatted_prompt)
        else:
            raise ValueError("Retriever is not defined. Please ensure the LLM and retrieval chain are set up first.")


 
 
pipeline = CSVPipeline()
pipeline.defineLLM("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", "Gemma2-9b-It")
pipeline.retrieval()
 
print(pipeline.answer("Give me the price and the difference between price of sku id 47-63-621-64 and sku id 47-65-121-30"))

MATCH (p1:Product {sku: "47-63-621-64"}), (p2:Product {sku: "47-65-121-30"}) RETURN p1.price AS price1, p2.price AS price2, p1.price - p2.price AS price_differenceThe price of sku id 47-63-621-64 is 3.11 and the price of sku id 47-65-121-30 is 2.74. The price difference is 0.37. 
None
